# Lab 8 Application layer: CDN and DNS

### Before you start please implement your name-Student_ID again and push it!!!

## Content Distribution Network

The key idea of CDN is to replicate popular Websites across many machines and use Pull && Push to combine cache and replication. Currently CDN is used so commonly that user can't "feel" the replication's existence. 

Let's see one example(refed from PKU Computer Network-summer school):

According to Google, the distance between Cambridge,UK to Nanjing,CN is 8916km and 8224km for Melbourne,AU and Nanjing,CN.

Let's ping the Cambridge U and monash U to get the RTT:

In [33]:
!ping www.cam.ac.uk -c 3

PING www.cam.ac.uk (128.232.132.8) 56(84) bytes of data.
64 bytes from tm-128-232-132-8.tm.uis.cam.ac.uk (128.232.132.8): icmp_seq=1 ttl=36 time=304 ms
64 bytes from tm-128-232-132-8.tm.uis.cam.ac.uk (128.232.132.8): icmp_seq=2 ttl=36 time=295 ms
64 bytes from tm-128-232-132-8.tm.uis.cam.ac.uk (128.232.132.8): icmp_seq=3 ttl=36 time=312 ms

--- www.cam.ac.uk ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2002ms
rtt min/avg/max/mdev = 295.259/303.740/312.304/6.958 ms


If you run this in Nanjing the RTT bt Cambridge U and NJU is about 290ms, however

In [41]:
!ping www.monash.edu -c 3

PING monash.squizedge.net (176.32.95.209) 56(84) bytes of data.
^C

--- monash.squizedge.net ping statistics ---
3 packets transmitted, 0 received, 100% packet loss, time 2030ms



So RTT this time is 44ms. But hold on, the distance between NJU and Monash U is 8224km, which means even light travel from NJU to Monash then back NJU will spend at least:

In [3]:
str(2*(8224/300000)*1000)+"ms"

'54.82666666666667ms'

Well what happened? Let's find out using traceroute

In [4]:
!traceroute www.cam.ac.uk

traceroute to www.cam.ac.uk (128.232.132.8), 30 hops max, 60 byte packets
 1  _gateway (114.212.80.1)  2.025 ms  1.858 ms  2.136 ms
 2  * * 172.28.255.13 (172.28.255.13)  1.595 ms
 3  * * *
 4  * * *
 5  211.65.207.73 (211.65.207.73)  3.384 ms  3.322 ms  3.404 ms
 6  101.4.112.65 (101.4.112.65)  2.578 ms 101.4.116.105 (101.4.116.105)  2.200 ms 101.4.116.222 (101.4.116.222)  2.038 ms
 7  101.4.112.42 (101.4.112.42)  17.280 ms  16.555 ms  16.257 ms
 8  101.4.117.30 (101.4.117.30)  35.918 ms 101.4.112.61 (101.4.112.61)  15.693 ms  15.749 ms
 9  101.4.117.38 (101.4.117.38)  32.976 ms  32.939 ms  32.941 ms
10  101.4.112.69 (101.4.112.69)  34.225 ms  34.029 ms 101.4.112.1 (101.4.112.1)  30.147 ms
11  101.4.114.194 (101.4.114.194)  37.102 ms 101.4.114.170 (101.4.114.170)  36.944 ms 101.4.113.109 (101.4.113.109)  32.418 ms
12  101.4.114.174 (101.4.114.174)  31.401 ms 210.25.189.65 (210.25.189.65)  34.856 ms  34.879 ms
13  210.25.189.65 (210.25.189.65)  32.476 ms 210.25.187.46 (210.25.187.46)  

In [5]:
!traceroute www.monash.edu.au

traceroute to www.monash.edu.au (176.32.95.209), 30 hops max, 60 byte packets
 1  _gateway (114.212.80.1)  1.533 ms  1.202 ms  1.318 ms
 2  172.28.255.13 (172.28.255.13)  0.912 ms * *
 3  * * *
 4  218.94.142.1 (218.94.142.1)  6.105 ms  3.431 ms  3.351 ms
 5  218.2.131.9 (218.2.131.9)  3.337 ms  3.298 ms  3.268 ms
 6  * * *
 7  218.2.182.33 (218.2.182.33)  4.773 ms  7.270 ms  6.425 ms
 8  202.97.92.21 (202.97.92.21)  15.438 ms  16.332 ms  15.410 ms
 9  * * *
10  * * *
11  * * *
12  * * *
13  * * *
14  * * *
15  * * *
16  * * *
17  * * *
18  * * *
19  * * *
20  * * *
21  * * *
22  * * *
23  * * *
24  * * *
25  * * *
26  * * *
27  * * *
28  * * *
29  * * *
30  * * *


You can use https://ipinfo.io/ to see who hosts this IP address. And 176.32.95.209: "This IP is hosted by Amazon.com, Inc. (AS16509) and located in the country Japan Tokyo."

Using CDN, we actually visit the replication server in Japan(1956km) instead of the one in Melbourne.

## Domain name system

DNS solves a bothering question that how to map from host name to IP address.

The key idea is hierarchy namespace with recursive query and iterative query.

Let's find out how it works,
###### The following is refed from http://www.ruanyifeng.com/blog/2016/06/dns.html

In [6]:
!ping www.nju.edu.cn -c1

PING www.nju.edu.cn (202.119.32.7) 56(84) bytes of data.
64 bytes from logic.nju.edu.cn (202.119.32.7): icmp_seq=1 ttl=62 time=1.15 ms

--- www.nju.edu.cn ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 1.147/1.147/1.147/0.000 ms


#### Let's first "dig" out NJU's IP address

Here on my computer `127.0.0.53` is my local DNS server and yours is just after `;; SERVER:`

In [7]:
!dig www.nju.edu.cn


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> www.nju.edu.cn
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 743
;; flags: qr rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 65494
;; QUESTION SECTION:
;www.nju.edu.cn.			IN	A

;; ANSWER SECTION:
www.nju.edu.cn.		3599	IN	A	202.119.32.7

;; Query time: 0 msec
;; SERVER: 127.0.0.53#53(127.0.0.53)
;; WHEN: Wed Dec 18 16:51:23 CST 2019
;; MSG SIZE  rcvd: 59



127.0.0.53 is my NJU local DNS server and there are also many famous public DNS server like Google's 8.8.8.8 alidns, you can change DNS server with  
```
nameserver 8.8.8.8
nameserver 8.8.4.4
nameserver 114.114.114.114
nameserver 42.120.214.1
```

in /etc/resolv.conf

### BACKUP BACKUP BACKUP

Sometimes, we may face the problem about error opening some specific pages(i.e. Github). You can manually change hosts `$ sudo gedit /etc/hosts` or DNS `$ sudo gedit /etc/resolv.conf`.

Let's see how to "dig" NJU's IP address with Google's DNS server 8.8.8.8

In [8]:
!dig @8.8.8.8 www.nju.edu.cn


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> @8.8.8.8 www.nju.edu.cn
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 21075
;; flags: qr rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 512
;; QUESTION SECTION:
;www.nju.edu.cn.			IN	A

;; ANSWER SECTION:
www.nju.edu.cn.		2652	IN	A	202.119.32.7

;; Query time: 42 msec
;; SERVER: 8.8.8.8#53(8.8.8.8)
;; WHEN: Wed Dec 18 16:51:23 CST 2019
;; MSG SIZE  rcvd: 59



Usually the host name contains four parts: aa.bb.cc.dd www.nju.edu.cn where 
+ aa:host name
+ bb:second level domain
+ cc:top level domain
+ dd:root (A to M)

and we usually neglect the root dd for `.com` because the root is all same. 

Next, let's see how to "dig" NJU's IP address

Because 127.0.0.53 is my NJU's local DNS server, thus it return NJU's IP address within 1ms.

In [9]:
!dig +trace www.nju.edu.cn


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> +trace www.nju.edu.cn
;; global options: +cmd
;; Received 51 bytes from 127.0.0.53#53(127.0.0.53) in 0 ms



Let's see what Google DNS server will do,

In [10]:
!dig +trace @8.8.8.8 www.nju.edu.cn


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> +trace @8.8.8.8 www.nju.edu.cn
; (1 server found)
;; global options: +cmd
.			8777	IN	NS	a.root-servers.net.
.			8777	IN	NS	b.root-servers.net.
.			8777	IN	NS	c.root-servers.net.
.			8777	IN	NS	d.root-servers.net.
.			8777	IN	NS	e.root-servers.net.
.			8777	IN	NS	f.root-servers.net.
.			8777	IN	NS	g.root-servers.net.
.			8777	IN	NS	h.root-servers.net.
.			8777	IN	NS	i.root-servers.net.
.			8777	IN	NS	j.root-servers.net.
.			8777	IN	NS	k.root-servers.net.
.			8777	IN	NS	l.root-servers.net.
.			8777	IN	NS	m.root-servers.net.
.			8777	IN	RRSIG	NS 8 0 518400 20191230050000 20191217040000 22545 . dFKL9Cy3e7YS21tReoW8TvDhQ9vWg85R0yHbWFtInRl4O2HQM6eLEMJW bnKTvRl4DkjSnGQUHG43xwNIVYje8kRA9CxRPYCtwarVqf3hQTmOVz4H gnLTGaxmMcgZajpJJqPMHnZJUXQ/Iicc/fWb7cwgGu1SMmOowuUXhYba eEYfYXdGLJIues3aio8RY4ui4dVuYuq3yyJOK0hrtpp6F5DaAeTc34ij V1lk64PiEZzJDudKeqzBlDYnXKj1iGwzOBLv+liocDfbQT5naL32l26S 1xy7Q5iT64j8RHrva1RssD+tmtgXffI9z84aLgbjg/uuExXM8cAaf8vN +K/zqA==
;; 

You can see it first check `.` which contains (all a to m) root server, then ask them for `.cn` where a,b,c,... give the reply, then check for `edu.cn` and after that with `nju.edu.cn`. Finally we get NJU's IP address from local DNS server: dns.nju.edu.cn

And you can also directly ask NJU's dns server for IP address

In [11]:
!dig @dns.nju.edu.cn www.nju.edu.cn


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> @dns.nju.edu.cn www.nju.edu.cn
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 62564
;; flags: qr aa rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;www.nju.edu.cn.			IN	A

;; ANSWER SECTION:
www.nju.edu.cn.		3600	IN	A	202.119.32.7

;; Query time: 0 msec
;; SERVER: 202.119.32.6#53(202.119.32.6)
;; WHEN: Wed Dec 18 16:51:25 CST 2019
;; MSG SIZE  rcvd: 59



In book we know that (Name,Value,Type,TTL) is one DNS Resource Record where Type can be A,NS,CNAME,MX.

Use them with nslookup,

In [12]:
!nslookup -type=A www.nju.edu.cn

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
Name:	www.nju.edu.cn
Address: 202.119.32.7



In [13]:
!nslookup -type=NS nju.edu.cn

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
nju.edu.cn	nameserver = dns.nju.edu.cn.
nju.edu.cn	nameserver = ns.nju.edu.cn.

Authoritative answers can be found from:



In [14]:
!nslookup -type=NS edu.cn

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
edu.cn	nameserver = dns.edu.cn.
edu.cn	nameserver = ns2.cernet.net.
edu.cn	nameserver = dns2.edu.cn.
edu.cn	nameserver = ns2.cuhk.hk.
edu.cn	nameserver = deneb.dfn.de.

Authoritative answers can be found from:



In [15]:
!nslookup -type=NS cn

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
cn	nameserver = c.dns.cn.
cn	nameserver = f.dns.cn.
cn	nameserver = b.dns.cn.
cn	nameserver = g.dns.cn.
cn	nameserver = ns.cernet.net.
cn	nameserver = e.dns.cn.
cn	nameserver = d.dns.cn.
cn	nameserver = a.dns.cn.

Authoritative answers can be found from:



In [16]:
!nslookup -type=NS .

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
.	nameserver = l.root-servers.net.
.	nameserver = d.root-servers.net.
.	nameserver = a.root-servers.net.
.	nameserver = j.root-servers.net.
.	nameserver = h.root-servers.net.
.	nameserver = i.root-servers.net.
.	nameserver = g.root-servers.net.
.	nameserver = b.root-servers.net.
.	nameserver = c.root-servers.net.
.	nameserver = k.root-servers.net.
.	nameserver = m.root-servers.net.
.	nameserver = e.root-servers.net.
.	nameserver = f.root-servers.net.

Authoritative answers can be found from:



In [17]:
!nslookup -type=MX nju.edu.cn

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
nju.edu.cn	mail exchanger = 10 mxbiz2.qq.com.
nju.edu.cn	mail exchanger = 5 mxbiz1.qq.com.

Authoritative answers can be found from:



In [18]:
!nslookup -type=CNAME mail.nju.edu.cn

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
mail.nju.edu.cn	canonical name = ssl.exmail.qq.com.

Authoritative answers can be found from:



#### Now back to CDN, how DNS server know which replication is the best?
Let's See one example about facebook\

If I ask NJU's DNS server what's FBs IP address 

In [19]:
!dig +trace @dns.nju.edu.cn www.facebook.com


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> +trace @dns.nju.edu.cn www.facebook.com
; (1 server found)
;; global options: +cmd
.			61795	IN	NS	d.root-servers.net.
.			61795	IN	NS	k.root-servers.net.
.			61795	IN	NS	g.root-servers.net.
.			61795	IN	NS	a.root-servers.net.
.			61795	IN	NS	b.root-servers.net.
.			61795	IN	NS	e.root-servers.net.
.			61795	IN	NS	l.root-servers.net.
.			61795	IN	NS	c.root-servers.net.
.			61795	IN	NS	j.root-servers.net.
.			61795	IN	NS	f.root-servers.net.
.			61795	IN	NS	i.root-servers.net.
.			61795	IN	NS	m.root-servers.net.
.			61795	IN	NS	h.root-servers.net.
;; Received 239 bytes from 202.119.32.6#53(dns.nju.edu.cn) in 0 ms

www.facebook.com.	110	IN	A	46.82.174.68
;; Received 50 bytes from 2001:7fe::53#53(i.root-servers.net) in 32 ms



If I ask Alidns:

In [20]:
!dig @42.120.214.1 www.facebook.com


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> @42.120.214.1 www.facebook.com
; (1 server found)
;; global options: +cmd
;; connection timed out; no servers could be reached


In [21]:
!dig +trace @8.8.8.8 www.facebook.com


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> +trace @8.8.8.8 www.facebook.com
; (1 server found)
;; global options: +cmd
.			40615	IN	NS	e.root-servers.net.
.			40615	IN	NS	h.root-servers.net.
.			40615	IN	NS	l.root-servers.net.
.			40615	IN	NS	i.root-servers.net.
.			40615	IN	NS	a.root-servers.net.
.			40615	IN	NS	d.root-servers.net.
.			40615	IN	NS	c.root-servers.net.
.			40615	IN	NS	b.root-servers.net.
.			40615	IN	NS	j.root-servers.net.
.			40615	IN	NS	k.root-servers.net.
.			40615	IN	NS	g.root-servers.net.
.			40615	IN	NS	m.root-servers.net.
.			40615	IN	NS	f.root-servers.net.
.			40615	IN	RRSIG	NS 8 0 518400 20191230050000 20191217040000 22545 . dFKL9Cy3e7YS21tReoW8TvDhQ9vWg85R0yHbWFtInRl4O2HQM6eLEMJW bnKTvRl4DkjSnGQUHG43xwNIVYje8kRA9CxRPYCtwarVqf3hQTmOVz4H gnLTGaxmMcgZajpJJqPMHnZJUXQ/Iicc/fWb7cwgGu1SMmOowuUXhYba eEYfYXdGLJIues3aio8RY4ui4dVuYuq3yyJOK0hrtpp6F5DaAeTc34ij V1lk64PiEZzJDudKeqzBlDYnXKj1iGwzOBLv+liocDfbQT5naL32l26S 1xy7Q5iT64j8RHrva1RssD+tmtgXffI9z84aLgbjg/uuExXM8cAaf

In [22]:
!dig +trace @c.root-servers.net www.facebook.com


; <<>> DiG 9.11.5-P4-5.1ubuntu2.1-Ubuntu <<>> +trace @c.root-servers.net www.facebook.com
; (2 servers found)
;; global options: +cmd
.			518400	IN	NS	j.root-servers.net.
.			518400	IN	NS	f.root-servers.net.
.			518400	IN	NS	d.root-servers.net.
.			518400	IN	NS	e.root-servers.net.
.			518400	IN	NS	l.root-servers.net.
.			518400	IN	NS	m.root-servers.net.
.			518400	IN	NS	k.root-servers.net.
.			518400	IN	NS	i.root-servers.net.
.			518400	IN	NS	b.root-servers.net.
.			518400	IN	NS	c.root-servers.net.
.			518400	IN	NS	g.root-servers.net.
.			518400	IN	NS	a.root-servers.net.
.			518400	IN	NS	h.root-servers.net.
.			518400	IN	RRSIG	NS 8 0 518400 20191231050000 20191218040000 22545 . T34Au86AqnmoHLcxFSWIqAdAB+KFrFwg3jhMQZJ4d7TeH3fFz8S+BAbC xOkARhSjs+GRGH3U3nozo/clTuJec9qtW9H4WG2SxumMGUBvUIaWWai/ R25TNY8Yth3qgUB6HIySLZsn9/hSgsS4irtJdEbAUqwcNCoqBiey51h2 zcF77JuyWjs4rmQK1vm3YAivT3KuNhiN+LSxt4rs/P9Xz8xTyccWC3gI u0t6MPVVsHJDZB0gU8NXgbqe19lqIsza7qCKrmAZ8NYRpR8koC4QFyxu Cr08JGP7J+99otvNEPYqOGC1RNT

(Optimal)Answer: DNS + Anycast

### Of course, you can open wireshark(filter: dns) to capture these DNS packet when runing above commands.

These are some questions leaving for you

Q1 DNS ports?

Q2 DNS packet should contain Queries and Answers, how many of each?

Q3 whats the difference between query with `www.` and without ?

These are RTT for different DNS server,

In [23]:
!ping 114.114.114.114 -c5

PING 114.114.114.114 (114.114.114.114) 56(84) bytes of data.
64 bytes from 114.114.114.114: icmp_seq=1 ttl=90 time=2.61 ms
64 bytes from 114.114.114.114: icmp_seq=2 ttl=81 time=2.39 ms
64 bytes from 114.114.114.114: icmp_seq=3 ttl=65 time=2.34 ms
64 bytes from 114.114.114.114: icmp_seq=4 ttl=76 time=1.95 ms
64 bytes from 114.114.114.114: icmp_seq=5 ttl=93 time=1.92 ms

--- 114.114.114.114 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4006ms
rtt min/avg/max/mdev = 1.917/2.239/2.606/0.267 ms


In [24]:
!ping 8.8.8.8 -c5

PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=50 time=41.9 ms
64 bytes from 8.8.8.8: icmp_seq=2 ttl=50 time=43.8 ms
64 bytes from 8.8.8.8: icmp_seq=3 ttl=50 time=41.6 ms
64 bytes from 8.8.8.8: icmp_seq=4 ttl=50 time=44.1 ms
64 bytes from 8.8.8.8: icmp_seq=5 ttl=50 time=41.9 ms

--- 8.8.8.8 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 41.591/42.669/44.131/1.062 ms


In [25]:
!ping 8.8.4.4 -c5

PING 8.8.4.4 (8.8.4.4) 56(84) bytes of data.
64 bytes from 8.8.4.4: icmp_seq=1 ttl=50 time=41.1 ms
64 bytes from 8.8.4.4: icmp_seq=2 ttl=50 time=34.4 ms
64 bytes from 8.8.4.4: icmp_seq=3 ttl=50 time=36.7 ms
64 bytes from 8.8.4.4: icmp_seq=4 ttl=50 time=34.6 ms
64 bytes from 8.8.4.4: icmp_seq=5 ttl=50 time=39.0 ms

--- 8.8.4.4 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4006ms
rtt min/avg/max/mdev = 34.386/37.161/41.099/2.590 ms


In [26]:
!ping 4.2.2.2 -c5

PING 4.2.2.2 (4.2.2.2) 56(84) bytes of data.
64 bytes from 4.2.2.2: icmp_seq=1 ttl=52 time=234 ms
64 bytes from 4.2.2.2: icmp_seq=2 ttl=52 time=233 ms
64 bytes from 4.2.2.2: icmp_seq=3 ttl=52 time=233 ms
64 bytes from 4.2.2.2: icmp_seq=4 ttl=52 time=234 ms
64 bytes from 4.2.2.2: icmp_seq=5 ttl=52 time=234 ms

--- 4.2.2.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 233.267/233.607/234.337/0.379 ms


In [27]:
!ping dns.nju.edu.cn -c5

PING dns.nju.edu.cn (202.119.32.6) 56(84) bytes of data.
64 bytes from dns.nju.edu.cn (202.119.32.6): icmp_seq=1 ttl=62 time=0.526 ms
64 bytes from dns.nju.edu.cn (202.119.32.6): icmp_seq=2 ttl=62 time=0.425 ms
64 bytes from dns.nju.edu.cn (202.119.32.6): icmp_seq=3 ttl=62 time=0.473 ms
64 bytes from dns.nju.edu.cn (202.119.32.6): icmp_seq=4 ttl=62 time=0.422 ms
64 bytes from dns.nju.edu.cn (202.119.32.6): icmp_seq=5 ttl=62 time=0.414 ms

--- dns.nju.edu.cn ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4080ms
rtt min/avg/max/mdev = 0.414/0.452/0.526/0.042 ms


In [28]:
!ping www.alidns.com -c5

PING www.alidns.com (42.120.214.1) 56(84) bytes of data.
64 bytes from 42.120.214.1 (42.120.214.1): icmp_seq=1 ttl=42 time=13.7 ms
64 bytes from 42.120.214.1 (42.120.214.1): icmp_seq=2 ttl=42 time=12.9 ms
64 bytes from 42.120.214.1 (42.120.214.1): icmp_seq=3 ttl=42 time=13.2 ms
64 bytes from 42.120.214.1 (42.120.214.1): icmp_seq=4 ttl=42 time=13.4 ms
64 bytes from 42.120.214.1 (42.120.214.1): icmp_seq=5 ttl=42 time=13.1 ms

--- www.alidns.com ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4040ms
rtt min/avg/max/mdev = 12.872/13.235/13.682/0.273 ms


### Please git push your name-student_ID

### No more assignment and good luck with your finals

### Thank you